In [1]:
import os

root_logdir = os.path.join(os.curdir, "my_logs")


def get_run_logdir():
    import time
    run_id = time.strftime("run_%Y_%m_%d-%H_%M_%S")
    return os.path.join(root_logdir, run_id)

In [2]:
run_logdir = get_run_logdir()

In [3]:
from tensorflow import keras
import numpy as np
import tensorflow as tf

In [4]:
keras.backend.clear_session()
np.random.seed(42)
tf.random.set_seed(42)

In [5]:
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

housing = fetch_california_housing()

X_train_full, X_test, y_train_full, y_test = train_test_split(housing.data, housing.target)
X_train, X_valid, y_train, y_valid = train_test_split(X_train_full, y_train_full)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_valid = scaler.transform(X_valid)
X_test = scaler.transform(X_test)

In [6]:
checkpoint_cb = keras.callbacks.ModelCheckpoint("my_keras_model.h5")

In [7]:
model = keras.models.Sequential([
    keras.layers.Dense(30, activation="relu", input_shape=[8]),
    keras.layers.Dense(30, activation="relu"),
    keras.layers.Dense(1)
])
model.compile(loss="mse", optimizer=keras.optimizers.SGD(learning_rate=1e-3))

2022-03-07 16:01:51.272100: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-03-07 16:01:51.272215: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-03-07 16:01:51.272748: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


In [8]:
tensorboard_cb = keras.callbacks.TensorBoard(run_logdir)
history = model.fit(X_train, y_train, epochs=30,
                    validation_data=(X_valid, y_valid),
                    callbacks=[checkpoint_cb, tensorboard_cb])

2022-03-07 16:01:51.291354: I tensorflow/core/profiler/lib/profiler_session.cc:136] Profiler session initializing.
2022-03-07 16:01:51.291371: I tensorflow/core/profiler/lib/profiler_session.cc:155] Profiler session started.
2022-03-07 16:01:51.291392: I tensorflow/core/profiler/lib/profiler_session.cc:172] Profiler session tear down.
2022-03-07 16:01:51.310502: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2022-03-07 16:01:51.336116: I tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 3686400000 Hz


Epoch 1/30
 77/363 [=====>........................] - ETA: 0s - loss: 5.6346

2022-03-07 16:01:51.604119: I tensorflow/core/profiler/lib/profiler_session.cc:136] Profiler session initializing.
2022-03-07 16:01:51.604141: I tensorflow/core/profiler/lib/profiler_session.cc:155] Profiler session started.
2022-03-07 16:01:51.611195: I tensorflow/core/profiler/lib/profiler_session.cc:71] Profiler session collecting data.
2022-03-07 16:01:51.611718: I tensorflow/core/profiler/lib/profiler_session.cc:172] Profiler session tear down.
2022-03-07 16:01:51.612668: I tensorflow/core/profiler/rpc/client/save_profile.cc:137] Creating directory: ./my_logs/run_2022_03_07-16_01_49/train/plugins/profile/2022_03_07_16_01_51
2022-03-07 16:01:51.613127: I tensorflow/core/profiler/rpc/client/save_profile.cc:143] Dumped gzipped tool data for trace.json.gz to ./my_logs/run_2022_03_07-16_01_49/train/plugins/profile/2022_03_07_16_01_51/asgrim-arch-desktop.trace.json.gz
2022-03-07 16:01:51.614336: I tensorflow/core/profiler/rpc/client/save_profile.cc:137] Creating directory: ./my_logs/run

363/363 [==============================] - 1s 3ms/step - loss: 3.3288 - val_loss: 0.7796
Epoch 2/30
363/363 [==============================] - 1s 2ms/step - loss: 0.7660 - val_loss: 0.6453
Epoch 3/30
363/363 [==============================] - 1s 2ms/step - loss: 0.6220 - val_loss: 0.6001
Epoch 4/30
363/363 [==============================] - 1s 2ms/step - loss: 0.5769 - val_loss: 0.5717
Epoch 5/30
363/363 [==============================] - 1s 2ms/step - loss: 0.5414 - val_loss: 0.5476
Epoch 6/30
363/363 [==============================] - 1s 2ms/step - loss: 0.5346 - val_loss: 0.5303
Epoch 7/30
363/363 [==============================] - 1s 2ms/step - loss: 0.5126 - val_loss: 0.5157
Epoch 8/30
363/363 [==============================] - 1s 2ms/step - loss: 0.4916 - val_loss: 0.5045
Epoch 9/30
363/363 [==============================] - 1s 2ms/step - loss: 0.4959 - val_loss: 0.4955
Epoch 10/30
363/363 [==============================] - 1s 2ms/step - loss: 0.4825 - val_loss: 0.4883
Epoch 11/3

#### Fine tuning Neural Networks

In [12]:
keras.backend.clear_session()
np.random.seed(42)
tf.random.set_seed(42)

X_new = X_test[:3]

In [19]:
def build_model(n_hidden=1, n_neurons=30, learning_rate=3e-3, input_shape=[8]) -> keras.models.Sequential:
    model = keras.models.Sequential()
    model.add(keras.layers.InputLayer(input_shape=input_shape))

    for layer in range(n_hidden):
        model.add(keras.layers.Dense(n_neurons, activation="relu"))

    model.add(keras.layers.Dense(1))
    optimizer = keras.optimizers.SGD(lr=learning_rate)
    model.compile(loss="mse", optimizer=optimizer)
    return model

In [20]:
keras_reg = keras.wrappers.scikit_learn.KerasRegressor(build_model)

In [15]:
keras_reg.fit(X_train, y_train, epochs=100, validation_data=(X_valid, y_valid),
              callbacks=[keras.callbacks.EarlyStopping(patience=10)])
mse_test = keras_reg.score(X_test, y_test)
y_pred = keras_reg.predict(X_new)

mse_test

Epoch 1/100
363/363 [==============================] - 1s 1ms/step - loss: 1.6195 - val_loss: 0.7676
Epoch 2/100
363/363 [==============================] - 1s 1ms/step - loss: 0.8228 - val_loss: 0.6127
Epoch 3/100
363/363 [==============================] - 0s 1ms/step - loss: 0.5930 - val_loss: 0.5436
Epoch 4/100
363/363 [==============================] - 1s 2ms/step - loss: 0.5167 - val_loss: 0.5164
Epoch 5/100
363/363 [==============================] - 1s 1ms/step - loss: 0.4790 - val_loss: 0.4981
Epoch 6/100
363/363 [==============================] - 1s 2ms/step - loss: 0.4823 - val_loss: 0.4882
Epoch 7/100
363/363 [==============================] - 0s 1ms/step - loss: 0.4658 - val_loss: 0.4783
Epoch 8/100
363/363 [==============================] - 0s 960us/step - loss: 0.4570 - val_loss: 0.4724
Epoch 9/100
363/363 [==============================] - 0s 857us/step - loss: 0.4594 - val_loss: 0.4673
Epoch 10/100
363/363 [==============================] - 0s 1ms/step - loss: 0.4494 - va

-0.3601016700267792

In [16]:
keras.backend.clear_session()
np.random.seed(42)
tf.random.set_seed(42)

In [21]:
from scipy.stats import reciprocal
from sklearn.model_selection import RandomizedSearchCV

param_distribs = {
    "n_hidden": [0, 1, 2, 3],
    "n_neurons": np.arange(1, 100),
    "learning_rate": reciprocal(3e-4, 3e-2)
}

rnd_search_cv = RandomizedSearchCV(keras_reg, param_distribs, n_iter=10, cv=3)
rnd_search_cv.fit(X_train, y_train, epochs=100, validation_data=(X_valid, y_valid),
                  callbacks=[keras.callbacks.EarlyStopping(patience=10)])

Epoch 1/100
242/242 [==============================] - 1s 2ms/step - loss: 5.0981 - val_loss: 3.7234
Epoch 2/100
242/242 [==============================] - 0s 2ms/step - loss: 3.2334 - val_loss: 2.3744
Epoch 3/100
242/242 [==============================] - 0s 1ms/step - loss: 2.0405 - val_loss: 1.6558
Epoch 4/100
242/242 [==============================] - 0s 1ms/step - loss: 1.4471 - val_loss: 1.2639
Epoch 5/100
242/242 [==============================] - 0s 1ms/step - loss: 1.1425 - val_loss: 1.0449
Epoch 6/100
242/242 [==============================] - 0s 2ms/step - loss: 0.9518 - val_loss: 0.9214
Epoch 7/100
242/242 [==============================] - 0s 1ms/step - loss: 0.8706 - val_loss: 0.8466
Epoch 8/100
242/242 [==============================] - 0s 2ms/step - loss: 0.7805 - val_loss: 0.7992
Epoch 9/100
242/242 [==============================] - 0s 1ms/step - loss: 0.7380 - val_loss: 0.7675
Epoch 10/100
242/242 [==============================] - 0s 1ms/step - loss: 0.7046 - val_lo

/home/asgrim/.pyenv/versions/miniconda3-latest/envs/book-learning/lib/python3.9/site-packages/sklearn/model_selection/_search.py:969: UserWarning: One or more of the test scores are non-finite: [-0.52192823 -0.76187469 -0.447192   -0.42339217         nan         nan
         nan -0.52029332 -0.33885426 -0.31665849]
  warnings.warn(


RuntimeError: Cannot clone object <tensorflow.python.keras.wrappers.scikit_learn.KerasRegressor object at 0x7ff679268580>, as the constructor either does not set or modifies parameter learning_rate

In [22]:
rnd_search_cv.best_params_

{'learning_rate': 0.003601683277857631, 'n_hidden': 3, 'n_neurons': 26}

In [23]:
rnd_search_cv.best_score_

-0.31665848692258197

In [30]:
learning_rate, n_hidden, n_neurons = rnd_search_cv.best_params_.values()

In [31]:
model = build_model(n_hidden, n_neurons, learning_rate)

In [32]:
model.save("best_reg_model.h5")

In [33]:
model.fit(X_train, y_train, epochs=100, validation_data=(X_valid, y_valid),
          callbacks=[keras.callbacks.EarlyStopping(patience=10)])

Epoch 1/100
363/363 [==============================] - 1s 2ms/step - loss: 1.8119 - val_loss: 0.7216
Epoch 2/100
363/363 [==============================] - 0s 1ms/step - loss: 0.7417 - val_loss: 0.6086
Epoch 3/100
363/363 [==============================] - 0s 1ms/step - loss: 0.5830 - val_loss: 0.5496
Epoch 4/100
363/363 [==============================] - 0s 1ms/step - loss: 0.5255 - val_loss: 0.5188
Epoch 5/100
363/363 [==============================] - 0s 1ms/step - loss: 0.4854 - val_loss: 0.4962
Epoch 6/100
363/363 [==============================] - 0s 1ms/step - loss: 0.4819 - val_loss: 0.4812
Epoch 7/100
363/363 [==============================] - 0s 1ms/step - loss: 0.4615 - val_loss: 0.4668
Epoch 8/100
363/363 [==============================] - 1s 1ms/step - loss: 0.4468 - val_loss: 0.4592
Epoch 9/100
363/363 [==============================] - 1s 2ms/step - loss: 0.4427 - val_loss: 0.4499
Epoch 10/100
363/363 [==============================] - 1s 2ms/step - loss: 0.4345 - val_lo

In [35]:
model.evaluate(X_test, y_test)

162/162 [==============================] - 0s 422us/step - loss: 0.3007


0.3007470369338989